## Building A Chatbot
In this ipynb We'll go over an example of how to design and implement an LLM-powered chatbot. This chatbot will be able to have a conversation and remember previous interactions.



In [2]:
import os
from dotenv import load_dotenv
load_dotenv() ## aloading all the environment variable

groq_api_key=os.getenv("GROQ_API_KEY")




In [3]:
from langchain_groq import ChatGroq
model=ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000021F96D4DF10>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000021F96DA9190>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [4]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi , My name is Krish and I am a Chief AI Engineer")])

AIMessage(content="Hello Krish, it's nice to meet you!  \n\nThat's a fascinating role!  I'm eager to learn more about your work as a Chief AI Engineer. What kind of projects are you currently involved in? Do you have any interesting challenges you're facing?\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 61, 'prompt_tokens': 22, 'total_tokens': 83, 'completion_time': 0.110909091, 'prompt_time': 0.002146736, 'queue_time': 0.24641271299999998, 'total_time': 0.113055827}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--51ef9c23-3fa9-4325-bedd-c326b74aec88-0', usage_metadata={'input_tokens': 22, 'output_tokens': 61, 'total_tokens': 83})

In [5]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="Hi , My name is Krish and I am a Chief AI Engineer"),
        AIMessage(content="Hello Krish! It's nice to meet you. \n\nAs a Chief AI Engineer, what kind of projects are you working on these days? \n\nI'm always eager to learn more about the exciting work being done in the field of AI.\n"),
        HumanMessage(content="Hey What's my name and what do I do?")
    ]
)

AIMessage(content='You are Krish, and you are a Chief AI Engineer.  \n\nI remember that from our earlier conversation! 😊 \n\nWhat can I help you with today, Krish?\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 38, 'prompt_tokens': 97, 'total_tokens': 135, 'completion_time': 0.069090909, 'prompt_time': 0.004332823, 'queue_time': 0.25213170700000004, 'total_time': 0.073423732}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--9afb00c4-048f-4083-8eec-6d2d7f42ac7b-0', usage_metadata={'input_tokens': 97, 'output_tokens': 38, 'total_tokens': 135})

### Message History
We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!

In [6]:
!pip install langchain_community

In [7]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store: 
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(model,get_session_history)

In [8]:
"""store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()                                 "     for understanding perpose  "
    return store[session_id]
"""

'store = {}\n\ndef get_session_history(session_id: str) -> BaseChatMessageHistory:\n    if session_id not in store:\n        store[session_id] = ChatMessageHistory()                                 "     for understanding perpose  "\n    return store[session_id]\n'

In [9]:
""" get_session_history("chet1")
 print(store)"""

' get_session_history("chet1")\n print(store)'

In [10]:
config={"configurable":{"session_id":"chat1"}}

In [11]:
response=with_message_history.invoke(
    [HumanMessage(content="Hi , My name is Krish and I am a Chief AI Engineer")],
    config=config
)

In [12]:
response.content

"Hi Krish, it's nice to meet you!  \n\nThat's a fascinating title.  What kind of AI work do you do as a Chief AI Engineer?  I'm always eager to learn more about how people are using AI in the real world.\n"

In [13]:
with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

AIMessage(content='Your name is Krish. 😊  I remember that from our introduction!  \n\nIs there anything else I can help you with?\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 94, 'total_tokens': 123, 'completion_time': 0.052727273, 'prompt_time': 0.004292433, 'queue_time': 0.243854675, 'total_time': 0.057019706}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--61f93b85-6267-4b72-b18d-64a1d235d747-0', usage_metadata={'input_tokens': 94, 'output_tokens': 29, 'total_tokens': 123})

In [14]:
## change the config-->session id
config1={"configurable":{"session_id":"chat2"}}
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

"As an AI, I have no memory of past conversations and do not know your name. If you'd like to tell me, I'm happy to know!\n"

In [15]:
response=with_message_history.invoke(
    [HumanMessage(content="Hey My name is John")],
    config=config1
)
response.content

"Hi John! It's nice to meet you. What can I do for you today?\n"

In [16]:
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

'Your name is John.  \n\nIs there anything else I can help you with, John? 😊  \n'

In [17]:
def show_history(session_id):
    if session_id in store:
        print(f"\n📜 History for session: {session_id}")
        for msg in store[session_id].messages:
            role = msg.type.capitalize()  # Human / AI
            print(f"{role}: {msg.content}")
    else:
        print(f"No history found for session '{session_id}'")

# Use it like this:
show_history("chat1")
show_history("chat2")



📜 History for session: chat1
Human: Hi , My name is Krish and I am a Chief AI Engineer
Ai: Hi Krish, it's nice to meet you!  

That's a fascinating title.  What kind of AI work do you do as a Chief AI Engineer?  I'm always eager to learn more about how people are using AI in the real world.

Human: What's my name?
Ai: Your name is Krish. 😊  I remember that from our introduction!  

Is there anything else I can help you with?


📜 History for session: chat2
Human: Whats my name
Ai: As an AI, I have no memory of past conversations and do not know your name. If you'd like to tell me, I'm happy to know!

Human: Hey My name is John
Ai: Hi John! It's nice to meet you. What can I do for you today?

Human: Whats my name
Ai: Your name is John.  

Is there anything else I can help you with, John? 😊  



In [18]:
"""import pandas as pd

def show_history_df(session_id):
    if session_id in store:
        messages = store[session_id].messages
        data = [{
            "Role": msg.type.capitalize(), 
            "Message": msg.content
        } for msg in messages]

        df = pd.DataFrame(data)
        print(f"\n📜 Chat History for session '{session_id}':")
        print(df)
    else:
        print(f"No history found for session '{session_id}'")

# Example:
show_history_df("chat1")
show_history_df("chat2")
"""

'import pandas as pd\n\ndef show_history_df(session_id):\n    if session_id in store:\n        messages = store[session_id].messages\n        data = [{\n            "Role": msg.type.capitalize(), \n            "Message": msg.content\n        } for msg in messages]\n\n        df = pd.DataFrame(data)\n        print(f"\n📜 Chat History for session \'{session_id}\':")\n        print(df)\n    else:\n        print(f"No history found for session \'{session_id}\'")\n\n# Example:\nshow_history_df("chat1")\nshow_history_df("chat2")\n'

### Prompt templates
Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. First, let's add in a system message with some custom instructions (but still taking messages as input). Next, we'll add in more input besides just the messages.

In [19]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant.Amnswer all the question to the nest of your ability"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain=prompt|model

In [20]:
chain.invoke({"messages":[HumanMessage(content="Hi My name is Krish")]})

AIMessage(content="Hi Krish, it's nice to meet you!\n\nI'm ready to answer your questions to the best of my ability.  \n\nWhat can I help you with today? 😄  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 43, 'prompt_tokens': 31, 'total_tokens': 74, 'completion_time': 0.078181818, 'prompt_time': 0.002401256, 'queue_time': 0.252233323, 'total_time': 0.080583074}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--a10adb2e-d64d-450b-b6eb-7c494cb86f9c-0', usage_metadata={'input_tokens': 31, 'output_tokens': 43, 'total_tokens': 74})

In [21]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history)

In [22]:
config = {"configurable": {"session_id": "chat3"}}
response=with_message_history.invoke(
    [HumanMessage(content="Hi My name is Krish")],
    config=config
)

response

AIMessage(content="Hi Krish, it's nice to meet you! 👋  \n\nI'm ready to answer your questions to the best of my ability. What can I do for you today? 😊  \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 42, 'prompt_tokens': 31, 'total_tokens': 73, 'completion_time': 0.076363636, 'prompt_time': 0.002351316, 'queue_time': 0.247368212, 'total_time': 0.078714952}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--f5b8c18f-04cf-4d87-9205-7a34ecdea913-0', usage_metadata={'input_tokens': 31, 'output_tokens': 42, 'total_tokens': 73})

In [23]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'Your name is Krish!  😊  I remember that from our introduction.  \n\n\nIs there anything else I can help you with?\n'

In [24]:
## Add more complexity

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [25]:
response=chain.invoke({"messages":[HumanMessage(content="Hi My name is Krish")],"language":"Hindi"})
response.content

'नमस्ते कृष्ण, \n\nमुझे आपकी मदद करने में खुशी हो रही है! \n\nआप मुझसे क्या जानना चाहते हैं?  \n\n'

Let's now wrap this more complicated chain in a Message History class. This time, because there are multiple keys in the input, we need to specify the correct key to use to save the chat history.

In [26]:
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [27]:
config = {"configurable": {"session_id": "chat4"}}
repsonse=with_message_history.invoke(
    {'messages': [HumanMessage(content="Hi,I am Krish")],"language":"Hindi"},
    config=config
)
repsonse.content

'नमस्ते कृष्ण! \n\nमुझे ख़ुशी है तुम्हारी मुलाक़ात करने में। आप मुझसे क्या जानना चाहते हैं? 😊 \n'

In [28]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="whats my name?")], "language": "Hindi"},
    config=config,
)

In [29]:
response.content

'आपका नाम कृष्ण है।  😊 \n'

### Managing the Conversation History
One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.
'trim_messages' helper to reduce how many messages we're sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages

In [30]:
from langchain_core.messages import SystemMessage,trim_messages
trimmer=trim_messages(
    max_tokens=45,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)
messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]
trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [31]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain=(
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    | prompt
    | model
    
)

response=chain.invoke(
    {
    "messages":messages + [HumanMessage(content="What ice cream do i like")],
    "language":"English"
    }
)
response.content

"As an AI, I don't have access to your personal information, including your ice cream preferences. \n\nWhat's your favorite flavor? 😊🍦  Maybe I can tell you some fun facts about it!\n"

In [32]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask")],
        "language": "English",
    }
)
response.content

'You asked "whats 2 + 2"  😊  \n\n\n\nLet me know if you want to try another one!'

In [33]:
## Lets wrap this in the MEssage History
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)
config={"configurable":{"session_id":"chat5"}}

In [34]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config,
)

response.content

"I don't have access to past conversations or any personal information about you, so I don't know your name.  Would you like to tell me? 😊  \n"

In [35]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="what math problem did i ask?")],
        "language": "English",
    },
    config=config,
)

response.content

"As a large language model, I have no memory of past conversations. If you'd like to ask me a math problem, I'm happy to help! 😊  \n\n"

In [ ]:
### from deepseek

### from deepseek

In [ ]:
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import AIMessage, HumanMessage
from typing import Dict, List

# Initialize the chatbot with conversation history tracking
class GroqChatBot:
    def __init__(self, groq_api_key: str, model_name: str = "Gemma2-9b-It"):
        self.llm = ChatGroq(groq_api_key=groq_api_key, model_name=model_name)
        self.conversation_histories: Dict[str, List] = {}
        
        # Set up the chat prompt template
        self.prompt_template = ChatPromptTemplate.from_messages([
            ("system", "You are a helpful AI assistant. You are talking to {user_name}."),
            *[("placeholder", "{chat_history}")],
            ("human", "{user_input}"),
        ])
        
        # Create the chain
        self.chain = self.prompt_template | self.llm | StrOutputParser()
    
    def get_history(self, user_id: str) -> List:
        """Get conversation history for a user"""
        return self.conversation_histories.get(user_id, [])
    
    def add_to_history(self, user_id: str, user_name: str, user_input: str, ai_response: str):
        """Add a conversation turn to the history"""
        if user_id not in self.conversation_histories:
            self.conversation_histories[user_id] = []
        
        self.conversation_histories[user_id].extend([
            HumanMessage(content=user_input),
            AIMessage(content=ai_response),
        ])
    
    def chat(self, user_id: str, user_name: str, user_input: str) -> str:
        """Process a user message and return AI response"""
        # Get the conversation history
        history = self.get_history(user_id)
        
        # Generate the AI response
        response = self.chain.invoke({
            "user_name": user_name,
            "chat_history": history,
            "user_input": user_input,
        })
        
        # Add this interaction to history
        self.add_to_history(user_id, user_name, user_input, response)
        
        return response

# Example usage
if __name__ == "__main__":
    # Replace with your actual Groq API key
    GROQ_API_KEY = "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"
    
    # Initialize the chatbot
    chatbot = GroqChatBot(GROQ_API_KEY)
    
    # Sample conversation
    user_id = "123"
    user_name = "Alice"
    
    while True:
        user_input = input(f"{user_name}: ")
        if user_input.lower() in ["exit", "quit"]:
            break
            
        response = chatbot.chat(user_id, user_name, user_input)
        print(f"AI: {response}")

AI: Hello Alice! 👋  What can I do for you today? 😊  


AI: I'm ready when you are!  Just let me know what's on your mind.  

Do you have a question, need help with a task, or just want to chat?  😄  




AI: Hi Om! It's nice to meet you. 😊 

What can I do for you today?  

AI: That's a great question, Om! I'm designed to be a helpful and informative AI assistant. Here are some things I can do:

* **Answer your questions:** I have access to a vast amount of information and can answer questions on a wide range of topics. 
* **Generate creative content:** I can write stories, poems, articles, and even code!
* **Summarize text:**  Need to get the gist of a long article or document? I can do that for you.
* **Translate languages:** I can translate text between many different languages.
* **Brainstorm ideas:** Stuck on a project? I can help you generate new ideas.
* **Have a conversation:**  I'm here to chat and have a friendly conversation on any topic you like.

Is there anything specific y

In [ ]:
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate

## Step 1: Set up the brain (AI)
groq_api_key = "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"  # Get this from Groq's website
ai_brain = ChatGroq(groq_api_key=groq_api_key, model_name="Gemma2-9b-It")

## Step 2: Create memory storage
conversation_memories = {}  # Will remember all user chats

## Step 3: Make the chat function
def chat(user_id, user_name, message):
    # Get this user's past chats (or start new if first time)
    if user_id not in conversation_memories:
        conversation_memories[user_id] = []
    
    # Prepare the prompt with memory
    prompt = ChatPromptTemplate.from_messages([
        ("system", f"You're talking to {user_name}. Be friendly and helpful."),
        *conversation_memories[user_id],  # Past messages
        ("human", message)  # New message
    ])
    
    # Get AI response
    chain = prompt | ai_brain
    response = chain.invoke({})
    
    # Remember this conversation
    conversation_memories[user_id].append(("human", message))
    conversation_memories[user_id].append(("ai", response.content))
    
    return response.content

## Let's try it!
if __name__ == "__main__":
    print("Chatbot ready! Type 'quit' to exit.")
    
    while True:
        # Get user info
        user_id = input("Enter your user ID (any number/name): ")
        user_name = input("Enter your name: ")
        
        # Chat loop
        while True:
            message = input(f"{user_name}: ")
            if message.lower() == 'quit':
                break
            
            response = chat(user_id, user_name, message)
            print(f"AI: {response}")

Chatbot ready! Type 'quit' to exit.
AI: Hey om2003joshi! 👋

What can I do for you today? 😊 I'm happy to answer any questions you have, help you brainstorm ideas, or just chat!  

Let me know how I can be of service. 😄 


AI: Hey there! 👋 

It seems like you're just hanging out.  What's on your mind? 🤔  

Maybe you want to:

* **Talk about something interesting you've read or seen?**
* **Get help with a creative project?**
* **Just have a friendly conversation?**

I'm here for whatever you'd like! 😊  




AI: Hey there! 👋  

It's great to hear from you!  What's up? 😊  Anything exciting happening?

AI: That's really kind of you to ask!  As an AI, I don't need help in the way a person might. 

But, you can definitely help me by:

* **Asking me questions!**  The more you interact with me, the better I become at understanding and responding.
* **Giving me feedback.**  If you think I've done a good job, let me know! If I could do better, tell me how. 
* **Just being friendly!**  A little con